In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar, Signal
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [ ]:
task = 'SMEG'
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[3]
noise_cov = 'baseline_cov'
start = 0.1
stop = 0.2
sfreq = 200
path = op.join(Analysis_path, task, 'meg', 'Stats', str(start)+'_'+str(stop))
filename = op.join(path, name+'-'+noise_cov+'.nc')
test_key = 'RS_vs_OM+N'
p_thresh = 0.05

In [ ]:
stats = xr.open_dataarray(filename, test_key)
stats.load()

## Order times by number of significant sources

In [ ]:
if np.where(stats.loc['p_val']<p_thresh)[1].any():
    t_signif = np.unique(np.where(stats.loc['p_val']<p_thresh)[1], return_counts=True)
    best_t = stats.time[t_signif[0][t_signif[1].argsort()][::-1]]
    print(t_signif)
    print(best_t)
else:
    warnings.warn('\nNo significant cluster.\n')

In [ ]:
t = best_t.values[3]
mask = stats.loc['p_val', :, t].values.ravel() < p_thresh
data = stats.loc['T_stat', :, t].values.ravel()[mask]
vertices = np.arange((stats.src.values[-1]+1)*2, dtype=int)[mask]
print('Significant clusters at t =', t, 's:', np.unique(stats.loc['clu_inds', :, t].values.ravel()[mask]))
cmax = np.max(np.abs(data))
clim = (-cmax, cmax)

## Visbrain

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both', sulcus=True)
b_obj.add_activation(data, vertices, hide_under=clim[0], hemisphere='both', smoothing_steps=4, cmap='bwr', clim=clim)
vb = Brain(brain_obj=b_obj, bgcolor=None)

In [ ]:
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, round(t, 3)), bgcolor=None)#, txtcolor='black')
vb.rotate(custom=(225, -45))
vb.show()
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)), transparent=True)

## Cluster analysis

In [ ]:
state = 'RS'
surface = 'ico4'
stc_file = op.join(Analysis_path, task, 'meg', 'SourceEstimate', 'fsaverage', 'Grand_average', '{}-{}-baseline_cov-surface_{}'.format(state, name, surface))
clusters = np.unique(stats.loc['clu_inds'].values[stats.loc['p_val'].values < p_thresh])
clusters

In [ ]:
clu = clusters[0]
vert_clu = np.stack(np.where(stats.loc['clu_inds'].values == clu)[0::2])
vert_clu

In [ ]:
stc = mne.read_source_estimate(stc_file)
data = stc.data.T[:,vert_clu[1]].T
data.shape

In [ ]:
#Signal(data[::10,:-1:6], sf=200, time=stats.time.values, xlabel='Time /s', ylabel='Amplitude /µV', grid_titles=['Source ' + str(k) for k in sorted(vert_clu[1])[::10]]).show()
#Signal(data[::10,:-1:6], form='butterfly', sf=200, time=stats.time.values, xlabel='Time /s', ylabel='Amplitude /µV', grid_titles=['Source ' + str(k) for k in sorted(vert_clu[1])[::10]]).show()
Signal(data.mean(axis=0), time=stc.times*1000, xlabel='Time /ms', ylabel='Amplitude /µV').show()

## Visbrain one hemi at a time

In [ ]:
if np.where(stats.loc['p_val', 'left']<p_thresh)[0].any():
    t_signif_lh = np.unique(np.where(stats.loc['p_val', 'left']<p_thresh)[0], return_counts=True)
    best_t_lh = stats.time[t_signif_lh[0][t_signif_lh[1].argsort()][::-1]]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

In [ ]:
if np.where(stats.loc['p_val', 'right']<p_thresh)[0].any():
    t_signif_rh = np.unique(np.where(stats.loc['p_val', 'right']<p_thresh)[0], return_counts=True)
    best_t_rh = stats.time[t_signif_rh[0][t_signif_rh[1].argsort()][::-1]]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

In [ ]:
t = float(best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0])
#t = float(best_t_rh[3])
#t = float(best_t_lh[1])
#t=0.07
print('t =', t, 's')
mask_lh = np.where(stats.loc['p_val', 'left', t] < p_thresh)
mask_rh = np.where(stats.loc['p_val', 'right', t] < p_thresh)
lh = np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, 0)
rh = np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, 0)

In [ ]:
cmax = np.max(np.abs(np.concatenate([lh, rh])))
clim = (-cmax, cmax)

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both', sulcus=True)

#b_obj.add_activation(data=np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, -np.inf), hide_under=clim[0], vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, -np.inf), hide_under=clim[0], vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=lh, vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=rh, vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)


vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

In [ ]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, t))
vb.rotate(custom=(225, -45))
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)))

In [ ]:
help(np.concatenate)

In [ ]:
np.stack((stats.src.values, stats.src.values))

In [ ]:
[d for d in data]

In [ ]:
data = stats.loc['T_stat', :, t].values
pval = stats.loc['p_val', :, t].values
np.savetxt('T_stat_2.csv', data, delimiter=',')
np.savetxt('p_val_2.csv', pval, delimiter=',')

In [ ]:
stats.loc['p_val', :, t].values

In [ ]:
(stats.src.values[-1]+1)*2